<a href="https://colab.research.google.com/github/ErickPower/CPSC501-Assignment4/blob/master/part2/notMNISToptimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git init
!git remote add origin https://github.com/ErickPower/CPSC501-Assignment4.git
!git pull origin master

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 76 (delta 34), reused 11 (delta 4), pack-reused 0
Unpacking objects: 100% (76/76), done.
From https://github.com/ErickPower/CPSC501-Assignment4
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master


In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  print("Tensorflow already loaded.")
  pass

TensorFlow 2.x selected.


Upload notMNIST.npz

In [0]:
#from google.colab import files
#uploaded = files.upload()

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print("--Get data--")
with np.load("notMNIST.npz", allow_pickle=True) as f:
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']

print("--Process data--")
print(len(y_train))
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train, x_test = x_train.reshape(x_train.shape[0], 28,28,1), x_test.reshape(x_test.shape[0], 28,28,1)
 
print("--Make model--")
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16,3, input_shape=(28,28,1)),
  tf.keras.layers.MaxPool2D((2,2)),
  tf.keras.layers.Conv2D(32,3),
  tf.keras.layers.MaxPool2D((2,2)),
  #tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(600, activation='relu'),
  tf.keras.layers.Dense(400, activation='relu'),
  tf.keras.layers.Dense(200, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()

optim = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)

model.compile(optimizer=optim, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


print("--Fit model--")
model.fit(x_train, y_train, epochs=30, verbose=2)

print("--Evaluate model--")
model_loss, model_acc = model.evaluate(x_test,  y_test, verbose=2) 
print(f"Model Loss:    {model_loss:.2f}")
print(f"Model Accuray: {model_acc*100:.1f}%")

In [0]:
#Save Model to notMNIST.h5
model.save("notMNISTopt.h5")

In [0]:
#from google.colab import files
#files.download('notMNIST.h5')

My original model consists of 5 layers; 1 flatten layer (to reshape data from 28x28 to 1x784), and 4 Dense layers. The dense layers are of decreasing size outputs (600, 400, 200, 10), and the first 3 use the Rectified Linear Unit activation function.

The fourth dense layer uses softmax activation function, as this is best for outputting a prediction between 0 and 1 for each possible result. Since the output size of this layer is 10, it is producing a prediction for the input (image) to be classified into each possible category (A-J, 10 categories).

I used 30 epochs as this got to an acceptable level of accuracy (98% train, 94% test)

After drawing the letter H (in imageH.png), I tested the model on this image. It incorrectly labelled it as D.

#Changes:

Added 2 Convolution layers, and 2 max pooling layers, to train on smaller image patterns.
First convolution layer has output size of 16, second has output of 32.